In [1]:
import pandas as pd
import networkx as nx
import re
import unidecode
import dask as dd

In [2]:
RAW_DATA_DIR = "spotify.csv"

In [3]:
#standart read csv won't work, commas are in names!!! 
n = 4 #number of columns
df = pd.read_csv(RAW_DATA_DIR,
                 usecols=range(n),
                 lineterminator='\n',
                 header=None)

df = df[1:] #take the data less the header row
df.columns = ["user_id", "artistname", "trackname", "playlistname"] #set the header row as the df header

In [4]:
#df[['user_id','artistname','trackname','playlistname']] = \
#df[['user_id','artistname','trackname','playlistname']].astype(str)

In [5]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def clean_text(text):
    text = remove_emojis(text)
    text = unidecode.unidecode(text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    return text

In [7]:
# remove non ASCII characters and null characters
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df = df.replace(r'\W+', '', regex=True)

In [8]:
df.groupby("trackname").agg(
    Count=("user_id", "count"),
    
    UsrUniCnt=("user_id", "nunique"),
    UsrRat=("user_id", lambda x: x.nunique()/x.count()),

    ArtUniCnt=("artistname", "nunique"),
    ArtRat=("artistname", lambda x: x.nunique()/x.count()),

    PlyUniCnt=("playlistname", "nunique"),
    PlyRat=("playlistname", lambda x: x.nunique()/x.count()),
)

/tmp/ipykernel_2307/3587927400.py:8: RuntimeWarning: invalid value encountered in long_scalars
  ArtRat=("artistname", lambda x: x.nunique()/x.count()),
/tmp/ipykernel_2307/3587927400.py:11: RuntimeWarning: invalid value encountered in long_scalars
  PlyRat=("playlistname", lambda x: x.nunique()/x.count()),


,Count,UsrUniCnt,UsrRat,ArtUniCnt,ArtRat,PlyUniCnt,PlyRat
trackname,,,,,,,
,5242,920,0.175506,592,0.113824,999,0.190794
0,16,14,0.875000,8,0.533333,16,1.000000
00,4,4,1.000000,3,0.750000,4,1.000000
000,3,3,1.000000,2,0.666667,3,1.000000
0000,4,4,1.000000,3,0.750000,4,1.000000
...,...,...,...,...,...,...,...
zvenya,5,5,1.000000,1,0.200000,5,1.000000
zylophone,1,1,1.000000,1,1.000000,1,1.000000
zyzz,1,1,1.000000,1,1.000000,1,1.000000


In [9]:
df.groupby("artistname").agg(
    Count=("user_id", "count"),
    
    UsrUniCnt=("user_id", "nunique"),
    UsrRat=("user_id", lambda x: x.nunique()/x.count()),

    ArtUniCnt=("artistname", "nunique"),
    ArtRat=("artistname", lambda x: x.nunique()/x.count()),

    PlyUniCnt=("playlistname", "nunique"),
    PlyRat=("playlistname", lambda x: x.nunique()/x.count()),
)

/tmp/ipykernel_2307/1162960436.py:11: RuntimeWarning: invalid value encountered in long_scalars
  PlyRat=("playlistname", lambda x: x.nunique()/x.count()),


,Count,UsrUniCnt,UsrRat,ArtUniCnt,ArtRat,PlyUniCnt,PlyRat
artistname,,,,,,,
,6066,762,0.125618,1,0.000165,832,0.137158
000,1,1,1.000000,1,1.000000,1,1.000000
00005,1,1,1.000000,1,1.000000,1,1.000000
00017,1,1,1.000000,1,1.000000,1,1.000000
001,2,1,0.500000,1,0.500000,2,1.000000
...,...,...,...,...,...,...,...
zuco103,1,1,1.000000,1,1.000000,1,1.000000
zurdokmovimento,1,1,1.000000,1,1.000000,1,1.000000
zweitausendeinsTraum,1,1,1.000000,1,1.000000,1,1.000000


In [10]:
df.head(3)

,user_id,artistname,trackname,playlistname
1,9cc0cfd4d7d7885102480dd99e7a90d6,ElvisCostello,TheAngelsWannaWearMyRedShoes,HARDROCK2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,ElvisCostelloTheAttractions,WhatsSoFunnyBoutPeaceLoveAndUnderstanding,HARDROCK2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,TiffanyPage,7YearsTooLate,HARDROCK2010


In [11]:
G=nx.from_pandas_edgelist(df, 'artistname', 'trackname',['playlistname','user_id'])

In [ ]:
pos = nx.spring_layout(G)
nx.draw(G, pos = pos)

In [ ]:
import scipy

In [ ]:
scipy.__version__